In [63]:
import re
from datetime import datetime
import laws_ai.read_page as read_page
import laws_ai.helpers as helpers
import laws_ai.db as db
import laws_ai.find_authors as find_authors
import os
import json

path = os.path.dirname(os.path.abspath('app'))


class Mine:
    def __init__(self, path, law_code, log=True, save=True):
        self.pattern = re.compile(
            "\d{1,2} (jan(eiro)?|fev(ereiro)?|mar(ço)?|abr(il)?|mai(o)|jun(ho)?|jul(ho)?|ago(sto)?|set(embro)?|out(ubro)?|nov(embro)?|dez(embro)?) \d{4}")
        self.months = {'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04',
                       'mai': '05', 'jun': '06', 'jul': '07', 'ago': '08',
                       'set': '09', 'out': '10', 'nov': '11', 'dez': '12'}
        self.path = path
        self.law_code = law_code
        self.save = save
        self.log = log
        self.current_published_at = ''
        self.current_law_id = ''
        link = db.link_by_law_code(law_code)
        self.current_link_id = link["_id"]
        self.current_text_id = ''

        self.published_at = None

    def clean_text(self, text):
        text = " ".join(text.split()).lower()
        text = re.sub('<[^>]+>', '', text)
        text = text.split("(function")[0]
        return text

    def preamble(self, text):
        ret = ''
        text = self.clean_text(text)
        text = text.replace("decreta:", ":")
        text = text.replace("decreto:", ":")
        text = text.split(":")[0]
        text_splited = text.split("presidente da república")
        if len(text_splited) > 1:
            ret = "presidente da república" + text_splited[1]
        return ret

    def description(self, text):
        ret = ''
        text = self.clean_text(text)
        text = text.replace(" no "," nº ")
        text = text.split(" nº ")[1:-1]
        text = " nº ".join(text)
        text = text.split()[7:-1]
        text = " ".join(text)
        #text = " ".join(" ".join(text[1:-1]).split()[6:-2])
        text_splited = text.split("presidente da república")
        text_splited = ".".join(text_splited[0].split(".")[0:-1])
        if len(text_splited) > 0:
            ret = text_splited
        return ret

    def law_type_date_scope(self, text):
        scope = ''
        text = self.clean_text(text)
        text = text.replace(" no "," nº ")
        text = text.split(" nº ")
        if 'presidência da república' in text[0]:
            scope = 'federal'

        law_type = text[0].split()[-1]
        date = text[1]
        date = date.replace("º","")
        date = date.replace(" de ", " ").split()
        date = helpers.only_numbers(
            date[3]) + '-' + self.months[date[2][:3]] + '-' + date[1]

        return law_type, date, scope

    def extract_tables(self, all_data, indent=None):
        tables = all_data.find_all("table")

        all_tables = []

        for table in tables:
            rows = table.find_all("tr")
            headers = {}

            thead = table.find("thead")
            if thead:
                thead = thead.find_all("th")
                for i in range(len(thead)):
                    headers[i] = thead[i].text.strip().lower()
            data = []
            for row in rows:
                cells = row.find_all("td")
                if thead:
                    items = {}
                    for index in headers:
                        items[headers[index]] = cells[index].text
                else:
                    items = []
                    for index in cells:
                        text = index.text.strip()
                        text = text.replace("\n", " ")
                        text = " ".join(text.split())
                        items.append(text)
                data.append(items)
            tb_data = json.dumps(data, indent=indent, ensure_ascii=False)
            all_tables.append(tb_data)

            # -- Add item form Fact
        data_table = {
            'law': self.current_law_id,
            'law_code': self.law_code,
            'tables': all_tables
        }
        db.add_tables(data_table)

        return all_tables

    def law_texts(self, itens, save=False):
        article = 0
        last_article = 0
        chapter = ''
        last_chapter = ''
        paragraph = ''
        last_paragraph = ''
        item = ''
        last_item = ''
        point = ''
        last_point = ''

        for p in itens:
            if p == "" or p == "*" or len(p) < 15:
                continue

            extinct = False
            if re.findall('revogado', p.lower()):
                extinct = True

            # -- Find the chapter
            if(re.findall('CAPÍTULO*...........', p)):
                chapter = re.findall(
                    'CAPÍTULO*...........', p)[0].split(" ")[1]
                if(last_chapter != chapter):
                    last_chapter = chapter
                    article = article + 1
                    last_article = ''
                    last_paragraph = ''
                    last_item = ''
                    last_point = ''

            # -- Find the article
            if(re.findall("Art", p)):
                i = re.findall(r'\d+', p)
                if i:
                    article = int(i[0])
                    if(last_article != article):
                        last_article = article
                        last_paragraph = ''
                        last_item = ''
                        last_point = ''

            # -- define paragraphs
            if(re.findall('§ *...........', p)):
                paragraph = int(re.findall(r'\d+', p)[0])
                if(last_paragraph != paragraph):
                    last_paragraph = paragraph
                    last_item = ''
                    last_point = ''

            # -- define items
            if(re.findall(r"[MDCLXVI] -", p)):
                item = p.split(" - ")[0]
                item = item.replace("(Vigência) ", "")
                item = item.split(" ")[0]
                if(last_item != item):
                    last_item = item
                    last_point = ''

            # -- define alínea
            if(re.findall(r"[)] ", p[0:8])):
                point = p[0:8].split(") ")[0]
                if(last_point != point):
                    last_point = point

            data_text = {
                'law_code': self.law_code, 'published_at': self.published_at,
                'chapter': chapter, 'article': article,
                'paragraph': paragraph, 'item': item,
                'point': point, 'text': p, 'extinct': extinct
            }

            current_text_id = ''

            if save:
                current_text_id = db.add_text(data_text)

                # -- Add item form Fact
                fact = {
                    'law': self.current_law_id,
                    'link': self.current_link_id,
                    'text': current_text_id,
                    'date': self.current_published_at
                }
                db.add_published_fact(fact)

    def extract_data(self, save=False):
        preamble = ""
        scope = ""
        effective_at = None
        description = ""
        scope = ""

        data = read_page.open_htm_file(self.path)

        law_type, self.published_at, scope = self.law_type_date_scope(
            data.text)

        self.extract_tables(data)
        description = self.description(data.text)
        preamble = self.preamble(data.text)

        # LIMPEZA
        for table in data.findAll("table"):
            table.extract()

        for footer in data.findAll("footer"):
            footer.extract()

        for br in data.find_all("br"):
            br.replace_with("\n")

        text = data.text
        text = text.split("(function")[0]

        authors = find_authors.authors(text)

        text = text.replace("\xa0", " ")
        text = text.replace("\n\n\n", "|")
        text = text.replace("\n\n", "|")
        text = text.replace("Art. ", "|Art. ")
        text = text.replace("\n", " ")
        text = " ".join(text.split())

        reduce_text = text.split("República")
        reduce_text = reduce_text[:-1]
        text = " ".join(reduce_text) + " República"

        items = helpers.clean_list(text.split("|"))

        cleaned_items = []
        for i in items:
            if '<!--' in i:
                continue

            cleaned_items.append(i)

        items = cleaned_items

        if effective_at == None:
            effective_at = self.published_at

        published_at_arr = self.published_at.split("-")

        law_date = {
            "_id": self.published_at,
            'year': int(published_at_arr[0]),
            'month': int(published_at_arr[1]),
            'day': int(published_at_arr[2]),
            'date': datetime.strptime(self.published_at, "%Y-%m-%d")
        }

        law = {
            'law_code': self.law_code, 'published_at': self.published_at,
            'effective_at': effective_at, 'law_type': law_type,
            'scope': scope, 'description': description,
            'preamble': preamble, 'authors': authors
        }

        if save:
            self.current_published_at = db.add_date(law_date)
            self.current_law_id = db.add_law(law)

        # Read items
        self.law_texts(items, save)

        return law


In [64]:
law = 'L12240'
#'D10102'

mine_law = Mine(path + "/laws/" + law + ".htm" , law)
x = mine_law.extract_data(True) 
x


{'law_code': 'L12240',
 'published_at': '2010-05-20',
 'effective_at': '2010-05-20',
 'law_type': 'lei',
 'scope': 'federal',
 'description': 'conversão da medida provisória nº 477, de 2009 abre crédito extraordinário, em favor de diversos órgãos e entidades do poder executivo, nº valor global de r$ 18.191.723.573,00, e reduz o orçamento de investimento de diversas empresas nº valor global de r$ 5.736.743.280,00, para os fins que especifica',
 'preamble': 'presidente da república adotou a medida provisória nº 477, de 2009, que o congresso nacional aprovou, e eu, marco maia, primeiro vice-presidente no exercício da presidência da mesa do congresso nacional, para os efeitos do disposto no art. 62 da constituição federal, com a redação dada pela emenda constitucional nº 32, combinado com o art. 12 da resolução nº 1, de 2002-cn, promulgo a seguinte lei',
 'authors': ['DEPUTADO MARCO MAIA',
  'PRIMEIRO VICE-PRESIDENTE NO EXERCICIO DA',
  'PRESIDENCIA MESA CONGRESSO'],
 '_id': ObjectId('5e68

In [228]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from string import punctuation

from nltk.stem.porter import *
stemmer = PorterStemmer()
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer(language='portuguese')

 


stop_words = set(stopwords.words('portuguese') + list(punctuation))

import spacy

nlp = spacy.load("pt_core_news_sm")
    
    
law = 'D10179'
path1 = path + "/laws/" + law + ".htm"
data = read_page.open_htm_file(path1)
a = " ".join(data.text.split())
import re
v = re.sub('<[^>]+>', '', a)
v = v.split("(function")[0]
v = v.lower() 
text = v

#text = nlp(v)



#text = nlp(v)

# text_lema = []
# for word in text:
#     text_lema.append(word.lemma_)
     
# text_lema = []
# for token in text.split():
#     text_lema.append(stemmer.stem(token))

text = [palavra for palavra in text.split() if palavra not in stop_words]
  

" ".join(text)
 
    



'd10179 presidência república secretaria geral subchefia assuntos jurídicos decreto nº 10.179, 18 dezembro 2019 declara revogação, fins disposto art. 16 lei complementar nº 95, 26 fevereiro 1998, decretos normativos. remover texto compilado texto atualizado texto impressãoup down vigência presidente república uso atribuição confere art. 84, caput, inciso iv, constituição, tendo vista disposto art. 16 lei complementar nº 95, 26 fevereiro 1998, decreta art. 1º fica declarada revogação do: i decreto nº 2.348, 28 dezembro 1910; ii art. 120 art. 126 decreto nº 24.114, 12 abril 1934; iii decreto nº 59.316, 28 setembro 1966; iv decreto nº 60.460, 13 março 1967; v decreto nº 70.465, 27 abril 1972; vi decreto nº 76.954, 30 dezembro 1975; vii decreto nº 85.471, 10 dezembro 1980; viii decreto nº 85.776, 26 fevereiro 1981; ix decreto nº 87.056, 23 março 1982; x decreto nº 88.206, 29 março 1983; xi decreto nº 90.638, 10 dezembro 1984; xii decreto nº 90.639, 10 dezembro 1984; xiii decreto nº 93.449,

In [204]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from string import punctuation


sentencas = sent_tokenize(texto)
for i in sentencas:
    print("---> ", i)

#def prepara_texto(texto):
    #sentencas = sent_tokenize(texto)
    #print("<<--", sentencas, "-->>")
    #palavras = word_tokenize(texto.lower())
    #stop_words = set(stopwords.words('portuguese') + list(punctuation))
    
    #return [palavra for palavra in palavras if palavra not in stop_words]


#palavras_sem_stopwords = prepara_texto(texto)
#palavras_sem_stopwords = " ".join(palavras_sem_stopwords)
#palavras_sem_stopwords

--->  d10179 presidência da república secretaria geral subchefia para assuntos jurídicos decreto nº 10.179, de 18 de dezembro de 2019 declara a revogação, para os fins do disposto no art.
--->  16 da lei complementar nº 95, de 26 de fevereiro de 1998, de decretos normativos.
--->  não remover texto compilado texto atualizado texto para impressãoup down vigência o presidente da república , no uso da atribuição que lhe confere o art.
--->  84, caput, inciso iv, da constituição, e tendo em vista o disposto no art.
--->  16 da lei complementar nº 95, de 26 de fevereiro de 1998, decreta : art.
--->  1º fica declarada a revogação do: i - decreto nº 2.348, de 28 de dezembro de 1910; ii - art.
--->  120 ao art.
--->  126 do decreto nº 24.114, de 12 de abril de 1934; iii - decreto nº 59.316, de 28 de setembro de 1966; iv - decreto nº 60.460, de 13 de março de 1967; v - decreto nº 70.465, de 27 de abril de 1972; vi - decreto nº 76.954, de 30 de dezembro de 1975; vii - decreto nº 85.471, de 10 de

In [60]:
import nltk 
from nltk.tokenize import sent_tokenize
# tokenized_sent = sent_tokenize(texto)
# for i in tokenized_sent:
#     print('---')
#     print(i)
    
    
    
sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')
sentences = sent_tokenizer.tokenize(texto)
for sent in sentences:
    print("<<", sent, ">>")


<< DEL1682 Presidência da República Casa Civil Subchefia para Assuntos Jurídicos DECRETO-LEI Nº 1682, DE 7 DE MAIO DE 1979 Vide Decreto Lei nº 2397, de 1987 Reduz alíquotas do imposto sobre Produtos Industrializados, assegura manutenção e utilização de crédito, e dá outras providências O PRESIDENTE DA REPÚBLICA, usando das atribuições que lhe confere o artigo 55, item Il da Constituição, DECRETA: Art 1º - Ficam reduzidas a 0 (zero) as alíquotas do Imposto sobre Produtos Industrializados relativas às seguintes mercadorias da Tabela anexa ao Regulamento aprovado pelo Decreto nº 83263, de 09 de março de 1979, desdobrados os respectivos códigos sob a forma de destaque("ex"): CÓDIGO MERCADORIA ALÍQUOTA 87040000 CHASSIS COM MOTOR, DOS VEÍCULOS AUTOMÓVEIS CLASSIFICADOS NAS POSIÇÕES 8701 A 8703 9900 Outros "ex" Para todos os tipos de ônibus e micro ônibus 0 87050000 CARROÇARIAS PARA OS VEÍCULOS AUTOMÓVEIS CLASSIFICADOS NAS POSIÇÕES 8701 A 8703, INCLUSIVE AS CABINAS 0300 Carroçarias próprias pa

In [188]:
tests = [
    { 
        'url': 'Del0376.htm',
        'ret': ''
    },
    { 
       'url': 'Del0555.htm',
       'ret': ['A COSTA E SILVA', 'ANTONIO DELFIM NETTO', 'MARIO DAVID ANDREAZZA', 'ANTONIO DIAS LEITE JUNIOR', 'HELIO BELTRAO']
    },
    {
        'url': 'Del1685.htm',
        'ret': ['JOAO FIGUEIREDO', 'KARLOS RISCHBIETER']
    },
    {
        "url": 'Del1682.htm',
        "ret": ['JOAO FIGUEIREDO', 'KARLOS RISCHBIETER']
    },
    {
        'url': 'D10102.htm',
        'ret': ['JAIR MESSIAS BOLSONARO', 'BENTO ALBUQUERQUE']
    },
    {
        'url': 'D10179.htm',
        'ret': ['JAIR MESSIAS BOLSONARO', 'JORGE ANTONIO OLIVEIRA FRANCISCO']
    },
    {
        'url': 'Del0278.htm',
        'ret': ['H CASTELO BRANCO', 'OCTAVIO BULHOES']
    },
    {
        'url': 'Del0073.htm',
        'ret': ['H CASTELO BRANCO',  'EDUARDO LOPES RODRIGUES', 'SEVERO FAGUNDES GOMES', 'L G NASCIMENTO E SILVA', 'RAYMUNDO BRITTO', 'PAULO EGYDIO MARTINS', 'ROBERTO CAMPOS']
    },
    {
        'url': 'Del0164.htm',
        'ret': ['H CASTELO BRANCO', 'LUIZ MARCELLO MOREIRA AZEVEDO', 'OCTAVIO BULHOES', 'EDMAR SOUZA', 'JOAO GONCALVES SOUZA']
    },
    {
        'url': 'Del0208impressao.htm',
        'ret': ['H CASTELO BRANCO', 'OCTAVIO BULHOES', 'JUAREZ TAVORA', 'MAURO THIBAU', 'EDMAR SOUZA']
    },
    {
        'url': 'Del0067.htm',
        'ret': ['H CASTELO BRANCO', 'EDUARDO LOPES RODRIGUES', 'JUAREZ TAVORA', 'L G NASCIMENTO E SILVA']
    },
    {
        'url': 'Del0055impressao.htm',
        'ret': ['H CASTELO BRANCO', 'JURACY MAGALHAES', 'OCTAVIO BULHOES', 'JUAREZ TAVORA', 'RAYMUNDO MONIZ ARAGAO', 'EDUARDO GOMES', 'PAULO EGYDIO MARTINS', 'ROBERTO CAMPOS']
    },
    {
        'url': 'Del0456.htm',
        'ret': ['A COSTA E SILVA', 'ANTONIO DELFIM NETTO', 'MARIO DAVID ANDREAZZA']
    },
    {
        'url': 'Del1295.htm',
        'ret': ['EMILIO G MEDICI', 'ANTONIO DELFIM NETTO']
    },
    {
        'url': 'Del1554.htm',
        'ret': ['ERNESTO GEISEL', 'MARIO HENRIQUE SIMONSEN']
    },
    {
        'url': 'Del0900.htm',
        'ret': ['AUGUSTO HAMANN RADEMAKER GRUNEWALD', 'AURELIO LYRA TAVARES', 'MARCIO SOUZA E MELLO', 'LUIS ANTONIO GAMA E SILVA', 'JOSE MAGALHAES PINTO',
                'ANTONIO DELFIM NETTO', 'MARIO DAVID ANDREAZZA', 'IVO ARZUA PEREIRA', 'TARSO DUTRA', 'JARBAS G PASSARINHO', 'LEONEL MIRANDA', 'EDMUNDO MACEDO SOARES', 'ANTONIO DIAS LEITE JUNIOR', 'HELIO BELTRAO', 'JOSE COSTA CAVALCANTI', 'CARLOS F SIMAS']
    },
    {
        'url': 'Del0799.htm',
        'ret': ['A COSTA E SILVA', 'AUGUSTO HAMANN RADEMAKER GRUNEWALD',
                'AURELIO LYRA TAVARES', 'ANTONIO DELFIM NETTO', 'MARIO DAVID ANDREAZZA', 'MARCIO SOUZA E MELLO', 'JOSE FERNANDES LUNA', 'HELIO BELTRAO']
    },
    {
        'url': 'Del0561.htm',
        'ret': ['A COSTA E SILVA', 'MARIO DAVID ANDREAZZA']
    },
    {
        'url': 'Del0376.htm',
        'ret': ['A COSTA E SILVA', 'LUIS ANTONIO GAMA E SILVA', 'ANTONIO DELFIM NETTO', 'HELIO BELTRAO']
    },
    {
        'url': 'Del0355.htm',
        'ret': ['A COSTA E SILVA', 'ANTONIO DELFIM NETTO', 'AFONSO A LIMA']
    },
    {
        'url': 'Del0300impresssao.htm',
        'ret': ['GETULIO VARGAS', 'ARTHUR SOUZA COSTA']
    },
    {
        'url': 'Del0055.htm',
        'ret': ['H CASTELO BRANCO', 'JURACY MAGALHAES', 'OCTAVIO BULHOES', 'JUAREZ TAVORA', 'RAYMUNDO MONIZ ARAGAO', 'EDUARDO GOMES', 'PAULO EGYDIO MARTINS', 'ROBERTO CAMPOS']
    },
    {
        'url': 'Del0244.htm',
        'ret': ['H CASTELO BRANCO', 'JUAREZ TAVORA', 'PAULO EGYDIO MARTINS', 'ROBERTO CAMPOS', 'OCTAVIO BULHOES']
    },
    {
        'url': 'Del0398.htm',
        'ret': ['A COSTA E SILVA', 'ANTONIO DELFIM NETTO']
    },
    {
        'url': 'Del1106.htm',
        'ret': ['EMILIO G MEDICI', 'ANTONIO DELFIM NETTO', 'MARIO DAVID ANDREAZZA', 'L F CIRNE LIMA', 'MARCUS VINICIUS PRATINI MORAES', 'JOAO PAULO REIS VELLOSO', 'JOSE COSTA CAVALCANTI']
    },
    {
        'url': 'Del0414.htm',
        'ret': ['A COSTA E SILVA', 'ANTONIO DELFIM NETTO', 'MARIO DAVID ANDREAZZA']
    },
    {
        'url': 'Del0509.htm',
        'ret': ['A COSTA E SILVA', 'ANTONIO DELFIM NETTO', 'JARBAS G PASSARINHO', 'HELIO BELTRAO', 'CARLOS F SIMAS']
    },
    {
        'url': 'Del0456.htm',
        'ret': ['A COSTA E SILVA', 'ANTONIO DELFIM NETTO', 'MARIO DAVID ANDREAZZA']
    },
    {
        'url': 'Del0398.htm',
        'ret': ['A COSTA E SILVA', 'ANTONIO DELFIM NETTO']
    },
    {
        'url': 'Del0200.htm',
        'ret': ['H CASTELO BRANCO', 'CARLOS MEDEIROS SILVA',
                'ZILMAR ARARIPE MACEDO','ADEMAR QUEIROZ',
                'MANOEL PIO CORREA JUNIOR','OCTAVIO GOUVEIA BULHOES',
                'JUAREZ NASCIMENTO TAVORA',
                'SEVERO GOMES FAGUNDES',
                'RAIMUNDO MONIZ ARAGAO',
                'LUIZ GONZAGA NASCIMENTO SILVA',
                'EDUARDO GOMES',
                'RAIMUNDO BRITO',
                'MAURO THIBAU',
                'PAULO EGYDIO MARTINS','ROBERTO OLIVEIRA CAMPOS', 'JOAO GONCALVES SOUZA E']
    }
]

In [190]:

for i in tests:
    law_path = path + "/laws/" + i["url"]
    law_code =  i["url"].split(".htm")[0]
    mine_law = Mine(law_path, law_code)
    ret = mine_law.extract_data()
   
    if ret["authors"] == i["ret"]:
        print("OK", i["url"])
    else: 
        ret
            


OK Del0555.htm
OK Del1685.htm
OK Del1682.htm
OK D10102.htm
OK D10179.htm
OK Del0278.htm
OK Del0073.htm
OK Del0208impressao.htm
OK Del0067.htm
OK Del0055impressao.htm
OK Del0456.htm
OK Del1295.htm
OK Del1554.htm
OK Del0799.htm
OK Del0561.htm
OK Del0376.htm
OK Del0355.htm
OK Del0300impresssao.htm
OK Del0244.htm
OK Del0398.htm
OK Del1106.htm
OK Del0509.htm
OK Del0456.htm
OK Del0398.htm
OK Del0200.htm


In [137]:

from keras.preprocessing.text import one_hot
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import hashing_trick

import re

# --> Del0376
# --> Del0555
# OK Del0555.htm
# --> Del1685
# OK Del1685.htm
# --> Del1682
# OK Del1682.htm
# --> D10102
# OK D10102.htm
# --> D10179


law = 'D10102'
#'Del0561'
#'D10179'
path1 = path + "/laws/" + law + ".htm"
data = read_page.open_htm_file(path1)
#text = prepar_text(data.text)
text = data.text
#" ".join(data.text.split())
# text = re.sub('<[^>]+>', '', text)
# text = text.split("(function")[0]
# text = text.lower()  

# define punctuation
#punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~''' 

# no_punct = ""
# for char in text:
#     if char not in punctuations:
#         no_punct = no_punct + char
        
# text = no_punct

# import nltk 
# from string import punctuation
# nltk.download('stopwords')
# stopwords = nltk.corpus.stopwords.words('portuguese')
# stop_words = set(stopwords + list(punctuation))
 

# text = [palavra for palavra in text.split() if palavra not in stop_words]
# text = " ".join(text)
# text



# import nltk    
# from nltk import tokenize    
# palavras_tokenize = tokenize.word_tokenize(text, language='portuguese')   
# palavras_tokenize

# # tokenize the document
# result = text_to_word_sequence(text)
# print(result)
# print(result.word_counts)
# print(t.document_count)
# print(t.word_index)
# print(t.word_docs)



# # estimate the size of the vocabulary
# words = set(text_to_word_sequence(text))
# vocab_size = len(words)
# print(vocab_size)
# # integer encode the document
# result = one_hot(text, round(vocab_size*1.3))
# print(result)

 
# # estimate the size of the vocabulary
# words = set(text_to_word_sequence(text))
# vocab_size = len(words)
# print(vocab_size)
# # integer encode the document
# result = hashing_trick(text, round(vocab_size*1.3), hash_function='md5')
# print(result)



# from keras.preprocessing.text import Tokenizer
# # define 5 documents
# docs = ['Well done!',
# 		'Good work',
# 		'Great effort',
# 		'nice work',
# 		'Excellent!']
# # create the tokenizer
# t = Tokenizer()
# # fit the tokenizer on the documents
# t.fit_on_texts(docs)

# # summarize what was learned
# print(t.word_counts)
# print(t.document_count)
# print(t.word_index)
# print(t.word_docs)

In [154]:
import nltk 
from string import punctuation
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
stop_words = set(stopwords + list(punctuation))

def prepar_text(text):
    text = " ".join(data.text.split())
    text = re.sub('<[^>]+>', '', text)
    text = text.split("(function")[0]
    text = text.lower()
    text = [palavra for palavra in text.split() if palavra not in stop_words]
    text = " ".join(text)
    return text


def text_paragraphs(text):
    text = text.split("(function")[0]
    #text = text.replace("O PRESIDENTE DA REPÚBLICA", "\n\n\n\n O PRESIDENTE DA REPÚBLICA")
    text = text.split("\n\n\n\n")
    n_text = []
    for i in text:
        c = " ".join(i.replace("\n"," ").split())
        n_text.append(c)
    text = list(filter(None, n_text))  
        
    #text = " ".join(text.split())
    #text = re.sub('<[^>]+>', '', text)
    #text = text.split("(function")[0]
    #text = text.lower()
    #text = [palavra for palavra in text.split() if palavra not in stop_words]
    #text = " ".join(text)
    return text
 
def preamble(text):
    text = " ".join(data.text.split()).lower()
     
    text = re.sub('<[^>]+>', '', text)  
        
    text = text.split("(function")[0]
    text = text.replace("decreta:","decreto:")
    text = text.split("decreto:")[0]
    
    text = "o presidente da república" + text.split("o presidente da república")[1]
    return text

def description(text):
    text = " ".join(data.text.split()).lower()
    text = re.sub('<[^>]+>', '', text)  
    text = text.split("(function")[0]
    text = text.split(",")
    text = " ".join(" ".join(text[1:-1]).split()[6:-2])
    text = text.split("o presidente da república")[0]
    return text

def law_type_and_data(text):
    text = " ".join(data.text.split()).lower()
    text = text.split("(function")[0]
    text = text.split(",")
    law_type = text[0].split()[-3]
    date = text[1].replace(" de ", " ").split()
    date = helpers.only_numbers(
            date[2]) + '-' + self.months[date[1][:3]] + '-' + date[0]
         
    return [law_type, date]
    

preamble(text) 


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/developer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'o presidente da república, no uso da atribuição que lhe confere o art. 84, caput, inciso vi, alínea “a”, da constituição, e tendo em vista o disposto no art. 6º da lei nº 13.576, de 26 de dezembro de 2017, '

In [26]:
from keras.preprocessing.text import Tokenizer
# define 5 documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)
# summarize what was learned
print(t.word_counts)
print(t.document_count)
print(t.word_index)
print(t.word_docs)
# integer encode documents
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])
5
{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}
defaultdict(<class 'int'>, {'done': 1, 'well': 1, 'good': 1, 'work': 2, 'effort': 1, 'great': 1, 'nice': 1, 'excellent': 1})
[[0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
